In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

In [2]:
# Créez un générateur d'images pour charger et prétraiter les données
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    'C:/Users/taouf/USE_CASE_3/Photos', 
    target_size=(224, 224),  # Taille des images pour VGG16
    batch_size=32,
    class_mode='binary',  # Deux classes : reportage et studio
    subset='training'  # Utilisation pour l'entraînement
)

val_gen = datagen.flow_from_directory(
    'C:/Users/taouf/USE_CASE_3/Photos', 
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'  # Utilisation pour la validation
)


Found 1119 images belonging to 2 classes.
Found 279 images belonging to 2 classes.


In [3]:
# Charger le modèle pré-entraîné sans les couches supérieures
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Geler toutes les couches de base_model
base_model.trainable = False

# Ajouter des couches personnalisées
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Sortie binaire
])

# Compiler le modèle
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


In [4]:
# Entraîner les couches ajoutées (initial training)
history_initial = model.fit(
    train_gen,  # Votre générateur d'images d'entraînement
    validation_data=val_gen,  # Votre générateur d'images de validation
    epochs=10,  # Entraînement initial avec 5 epochs
    batch_size=32
)

Epoch 1/10


35/35 [==============================] - 193s 5s/step - loss: 0.6912 - accuracy: 0.5380 - val_loss: 0.6555 - val_accuracy: 0.7061
Epoch 2/10
35/35 [==============================] - 251s 7s/step - loss: 0.6409 - accuracy: 0.7122 - val_loss: 0.6100 - val_accuracy: 0.7670
Epoch 3/10
35/35 [==============================] - 522s 15s/step - loss: 0.6029 - accuracy: 0.7417 - val_loss: 0.5646 - val_accuracy: 0.7993
Epoch 4/10
35/35 [==============================] - 435s 13s/step - loss: 0.5664 - accuracy: 0.7766 - val_loss: 0.5285 - val_accuracy: 0.8065
Epoch 5/10
35/35 [==============================] - 309s 9s/step - loss: 0.5387 - accuracy: 0.7882 - val_loss: 0.5008 - val_accuracy: 0.8065
Epoch 6/10
35/35 [==============================] - 190s 5s/step - loss: 0.5141 - accuracy: 0.7891 - val_loss: 0.4749 - val_accuracy: 0.8244
Epoch 7/10
35/35 [==============================] - 188s 5s/step - loss: 0.4916 - accuracy: 0.7954 - val_loss: 0.4550 - val_accuracy: 0.8459
Epoch 8/1

In [5]:
test_loss, test_acc = model.evaluate(val_gen)
print(f"Test accuracy: {test_acc}")

9/9 [==============================] - 38s 4s/step - loss: 0.4102 - accuracy: 0.8495
Test accuracy: 0.8494623899459839


In [6]:
# Débloquer certaines couches (fine-tuning)
base_model.trainable = True

# Geler une partie des couches (par exemple, jusqu'à `block4_pool`)
for layer in base_model.layers[:15]:  # Les 15 premières couches restent gelées
    layer.trainable = False

# Recompiler le modèle avec un taux d'apprentissage plus faible
model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])


In [7]:
# Fine-tuning avec un faible taux d'apprentissage
history_finetune = model.fit(
    train_gen,  # Générateur d'entraînement
    validation_data=val_gen,  # Générateur de validation
    epochs=10,  # Limité à 5 epochs
    batch_size=32
)

Epoch 1/10
35/35 [==============================] - 211s 6s/step - loss: 0.3967 - accuracy: 0.8222 - val_loss: 0.3252 - val_accuracy: 0.8674
Epoch 2/10
35/35 [==============================] - 212s 6s/step - loss: 0.3121 - accuracy: 0.8606 - val_loss: 0.2737 - val_accuracy: 0.8961
Epoch 3/10
35/35 [==============================] - 216s 6s/step - loss: 0.2541 - accuracy: 0.9088 - val_loss: 0.2500 - val_accuracy: 0.9104
Epoch 4/10
35/35 [==============================] - 212s 6s/step - loss: 0.2081 - accuracy: 0.9258 - val_loss: 0.2350 - val_accuracy: 0.9032
Epoch 5/10
35/35 [==============================] - 212s 6s/step - loss: 0.1953 - accuracy: 0.9249 - val_loss: 0.2496 - val_accuracy: 0.8781
Epoch 6/10
35/35 [==============================] - 212s 6s/step - loss: 0.1576 - accuracy: 0.9500 - val_loss: 0.2482 - val_accuracy: 0.8996
Epoch 7/10
35/35 [==============================] - 213s 6s/step - loss: 0.1324 - accuracy: 0.9669 - val_loss: 0.2261 - val_accuracy: 0.8889
Epoch 8/10
35

In [8]:
test_loss, test_acc = model.evaluate(val_gen)
print(f"Test accuracy: {test_acc}")

9/9 [==============================] - 38s 4s/step - loss: 0.2014 - accuracy: 0.9140
Test accuracy: 0.9139785170555115


In [17]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Chemin vers le dossier contenant les images
folder_path = 'C:/Users/taouf/USE_CASE_3/Photos_Test/reportage_test'

def predict_image(img_path, model):
    # Charger l'image et la prétraiter
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalisation comme à l'entraînement

    # Faire la prédiction
    prediction = model.predict(img_array)
    return 'Reportage' if prediction[0] < 0.5 else 'Studio'

def predict_on_folder(folder_path, model):
    results = {'Studio': 0, 'Reportage': 0}  # Compteur pour chaque type
    total_images = 0  # Compteur pour le nombre total d'images
    
    for root, _, files in os.walk(folder_path):  # Parcourt le dossier et ses sous-dossiers
        for filename in files:
            if filename.endswith(('.jpg', '.jpeg', '.png')):  # Vérifie les extensions d'image
                img_path = os.path.join(root, filename)
                label = predict_image(img_path, model)
                results[label] += 1  # Incrémente le compteur pour le type prédit
                total_images += 1  # Incrémente le compteur total

    return results, total_images

# Prédictions sur toutes les images du dossier et des sous-dossiers
results, total_images = predict_on_folder(folder_path, model)

# Afficher les résultats
for filename, label in predictions:
    print(f"{filename}: {label}")

# Calcul des pourcentages
if total_images > 0:  # Éviter la division par zéro
    studio_percentage = (results['Studio'] / total_images) * 100
    reportage_percentage = (results['Reportage'] / total_images) * 100

    # Afficher les résultats
    print(f"Total d'images : {total_images}")
    print(f"Studio : {results['Studio']} ({studio_percentage:.2f}%)")
    print(f"Reportage : {results['Reportage']} ({reportage_percentage:.2f}%)")
else:
    print("Aucune image trouvée dans le dossier ou ses sous-dossiers.")


1/1 [==============================] - 0s 226ms/step


1/1 [==============================] - 0s 295ms/step


1/1 [==============================] - 0s 256ms/step


1/1 [==============================] - 0s 210ms/step
C:/Users/taouf/USE_CASE_3/Photos_Test/reportage_test\air hockey\1.jpg: Reportage
C:/Users/taouf/USE_CASE_3/Photos_Test/reportage_test\air hockey\2.jpg: Reportage
C:/Users/taouf/USE_CASE_3/Photos_Test/reportage_test\air hockey\3.jpg: Studio
C:/Users/taouf/USE_CASE_3/Photos_Test/reportage_test\air hockey\4.jpg: Reportage
C:/Users/taouf/USE_CASE_3/Photos_Test/reportage_test\air hockey\5.jpg: Reportage
C:/Users/taouf/USE_CASE_3/Photos_Test/reportage_test\ampute football\1.jpg: Reportage
C:/Users/taouf/USE_CASE_3/Photos_Test/reportage_test\ampute football\2.jpg: Reportage
C:/Users/taouf/USE_CASE_3/Photos_Test/reportage_test\ampute football\3.jpg: Reportage
C:/Users/taouf/USE_CASE_3/Photos_Test/reportage_test\ampute football\4.jpg: Reportage
C:/Users/taouf/USE_CASE_3/Photos_Test/reportage_test\ampute football\5.jpg: Reportage
C:/Users/taouf/USE_CASE_3/Photos_Test/reportage_test\archery\1.jpg: Studio
C:/Users/taouf/USE_CASE_3/Photos_Test/re

In [20]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Chemin vers le dossier contenant les images
folder_path = 'C:/Users/taouf/USE_CASE_3/Photos_Test/studio_test'

def predict_image(img_path, model):
    # Charger l'image et la prétraiter
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalisation comme à l'entraînement

    # Faire la prédiction
    prediction = model.predict(img_array)
    return 'Reportage' if prediction[0] < 0.5 else 'Studio'

def predict_on_folder(folder_path, model):
    results = {'Studio': 0, 'Reportage': 0}  # Compteur pour chaque type
    predictions = []  # Stocker les prédictions individuelles
    total_images = 0  # Compteur pour le nombre total d'images

    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png')):  # Vérifie les extensions d'image
            img_path = os.path.join(folder_path, filename)
            label = predict_image(img_path, model)
            results[label] += 1  # Incrémente le compteur pour le type prédit
            predictions.append((filename, label))  # Ajoute la prédiction individuelle
            total_images += 1  # Incrémente le compteur total
            
    return results, predictions, total_images

# Prédictions sur toutes les images du dossier
results, predictions, total_images = predict_on_folder(folder_path, model)

# Afficher les prédictions individuelles
for filename, label in predictions:
    print(f"{filename}: {label}")

# Calcul des pourcentages
if total_images > 0:  # Éviter la division par zéro
    studio_percentage = (results['Studio'] / total_images) * 100
    reportage_percentage = (results['Reportage'] / total_images) * 100

    # Afficher les résultats globaux
    print(f"Total d'images : {total_images}")
    print(f"Studio : {results['Studio']} ({studio_percentage:.2f}%)")
    print(f"Reportage : {results['Reportage']} ({reportage_percentage:.2f}%)")
else:
    print("Aucune image trouvée dans le dossier.")

1/1 [==============================] - 0s 253ms/step


1/1 [==============================] - 0s 412ms/step


1/1 [==============================] - 1s 546ms/step


1/1 [==============================] - 0s 350ms/step
533.jpg: Reportage
535.jpg: Studio
536.jpg: Reportage
537.jpg: Reportage
538.jpg: Studio
539.jpg: Reportage
54.jpg: Studio
540.jpg: Studio
541.jpg: Studio
542.jpg: Reportage
543.jpg: Studio
544.jpg: Reportage
55.jpg: Studio
56.jpg: Reportage
560.jpg: Reportage
561.jpg: Studio
562.jpg: Studio
563.png: Studio
564.jpg: Studio
565.jpg: Reportage
566.jpg: Reportage
567.png: Studio
568.jpg: Studio
569.jpg: Studio
57.jpg: Reportage
570.jpg: Reportage
571.jpg: Studio
572.jpg: Reportage
573.jpg: Studio
574.jpg: Studio
575.jpg: Studio
576.jpg: Reportage
577.jpg: Studio
578.jpg: Reportage
579.jpg: Studio
58.jpg: Studio
580.jpg: Reportage
581.jpg: Reportage
582.jpg: Studio
583.jpg: Studio
584.png: Studio
585.jpg: Studio
586.jpg: Reportage
587.jpg: Reportage
588.jpg: Studio
589.jpg: Studio
59.jpg: Studio
590.jpg: Studio
591.jpg: Reportage
592.jpg: Studio
593.png: Reportage
594.jpg: Studio
595.jpg: Reportage
596.jpg: Studio
597.jpg: Studio
598.jpg

In [21]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Chemin vers le dossier contenant les images
folder_path = 'C:/Users/taouf/USE_CASE_3/Photos_Test'

def predict_image(img_path, model):
    # Charger l'image et la prétraiter
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalisation comme à l'entraînement

    # Faire la prédiction
    prediction = model.predict(img_array)
    return 'Reportage' if prediction[0] < 0.5 else 'Studio'

def predict_on_folder(folder_path, model):
    results = {'Studio': 0, 'Reportage': 0}  # Compteur pour chaque type
    total_images = 0  # Compteur pour le nombre total d'images
    
    for root, _, files in os.walk(folder_path):  # Parcourt le dossier et ses sous-dossiers
        for filename in files:
            if filename.endswith(('.jpg', '.jpeg', '.png')):  # Vérifie les extensions d'image
                img_path = os.path.join(root, filename)
                label = predict_image(img_path, model)
                results[label] += 1  # Incrémente le compteur pour le type prédit
                total_images += 1  # Incrémente le compteur total

    return results, total_images

# Prédictions sur toutes les images du dossier et des sous-dossiers
results, total_images = predict_on_folder(folder_path, model)

# Afficher les résultats
for filename, label in predictions:
    print(f"{filename}: {label}")

# Calcul des pourcentages
if total_images > 0:  # Éviter la division par zéro
    studio_percentage = (results['Studio'] / total_images) * 100
    reportage_percentage = (results['Reportage'] / total_images) * 100

    # Afficher les résultats
    print(f"Total d'images : {total_images}")
    print(f"Studio : {results['Studio']} ({studio_percentage:.2f}%)")
    print(f"Reportage : {results['Reportage']} ({reportage_percentage:.2f}%)")
else:
    print("Aucune image trouvée dans le dossier ou ses sous-dossiers.")


1/1 [==============================] - 0s 245ms/step


1/1 [==============================] - 0s 221ms/step


1/1 [==============================] - 0s 232ms/step


1/1 [==============================] - 0s 271ms/step


1/1 [==============================] - 0s 281ms/step


1/1 [==============================] - 0s 246ms/step


1/1 [==============================] - 0s 267ms/step
533.jpg: Reportage
535.jpg: Studio
536.jpg: Reportage
537.jpg: Reportage
538.jpg: Studio
539.jpg: Reportage
54.jpg: Studio
540.jpg: Studio
541.jpg: Studio
542.jpg: Reportage
543.jpg: Studio
544.jpg: Reportage
55.jpg: Studio
56.jpg: Reportage
560.jpg: Reportage
561.jpg: Studio
562.jpg: Studio
563.png: Studio
564.jpg: Studio
565.jpg: Reportage
566.jpg: Reportage
567.png: Studio
568.jpg: Studio
569.jpg: Studio
57.jpg: Reportage
570.jpg: Reportage
571.jpg: Studio
572.jpg: Reportage
573.jpg: Studio
574.jpg: Studio
575.jpg: Studio
576.jpg: Reportage
577.jpg: Studio
578.jpg: Reportage
579.jpg: Studio
58.jpg: Studio
580.jpg: Reportage
581.jpg: Reportage
582.jpg: Studio
583.jpg: Studio
584.png: Studio
585.jpg: Studio
586.jpg: Reportage
587.jpg: Reportage
588.jpg: Studio
589.jpg: Studio
59.jpg: Studio
590.jpg: Studio
591.jpg: Reportage
592.jpg: Studio
593.png: Reportage
594.jpg: Studio
595.jpg: Reportage
596.jpg: Studio
597.jpg: Studio
598.jpg